In [14]:
import random
import string
import os

In [15]:
def generate_random_files(byte_size: int):
    random_text: str = ''.join(random.choices(string.ascii_letters + string.digits + string.punctuation + ' ', k = byte_size))
    return random_text

def write_to_file(directory: str, filename: str, byte_size: int):
    if not os.path.exists(directory):
        os.makedirs(directory)

    file_path = os.path.join(directory,filename)
    random_text = generate_random_files(byte_size)

    with open(file_path,'w') as file:
        file.write(random_text)

    print(f'File "{filename}" with size {byte_size} bytes has been generated.')

In [16]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from binascii import hexlify
from numpy import array, mean, var, std, sqrt

import timeit, secrets

In [17]:
def aes_encrypt_file(path: str, key: int, new_path: str = None) -> tuple[int, int]:
    # Generate a random IV (16 bytes for AES)
    iv = secrets.token_bytes(16)

    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    encryptor = cipher.encryptor()

    # Read the plaintext file in binary mode
    with open(path, "rb") as plaintext_file:
        plaintext = plaintext_file.read()

    # Make sure the plaintext length is a multiple of block size (AES block size is 16 bytes)
    padding_length = 16 - len(plaintext) % 16
    padded_plaintext = plaintext + bytes([padding_length]) * padding_length

    # Encrypt the padded plaintext
    ct = encryptor.update(padded_plaintext) + encryptor.finalize()

    # Write the ciphertext to a new file
    with open(path if new_path == None else new_path + ".bin", "wb") as cphFile:
        cphFile.write(iv)  # Prepend IV to the ciphertext for decryption later
        cphFile.write(ct)

    # print(f"Encryption successful! File saved as: {new_path or path}")
    return hexlify(key), iv

def aes_decrypt_file(path: str, key: int, new_path: str = None) -> None:
    # Read the ciphertext file
    with open(path, "rb") as cphFile:
        iv = cphFile.read(16)  # Extract the IV (first 16 bytes)
        ct = cphFile.read()    # Read the rest as ciphertext

    # Create the cipher
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    decryptor = cipher.decryptor()

    # Decrypt the ciphertext
    padded_plaintext = decryptor.update(ct) + decryptor.finalize()

    # Remove padding (last byte value determines padding length)
    padding_length = padded_plaintext[-1]
    plaintext = padded_plaintext[:-padding_length]

    # Write the decrypted plaintext to a new file
    with open(path[:-4] if new_path is None else new_path, "wb") as plaintext_file:
        plaintext_file.write(plaintext)

    # print(f"Decryption successful! File saved as: {new_path or path}")

In [18]:
def test_aes(files: str, times: int, repeat: int) -> tuple[tuple[float, float, float], tuple[float, float, float]]:
    results: list[tuple[tuple[float, float, float], tuple[float, float, float]]] = []
    for file in files:
        key: int = os.urandom(32)
        encrypt_trials = array(timeit.repeat(
                lambda: aes_encrypt_file(
                    file,
                    key,
                    new_path = os.path.join(os.path.dirname(file), "encrypted_" + os.path.basename(file))
                ),
                number = times,
                repeat = repeat
            ))
        decrypt_trials = array(timeit.repeat(
                lambda: aes_encrypt_file(
                    os.path.join(os.path.dirname(file), "encrypted_" + os.path.basename(file) + ".bin"),
                    key,
                    new_path = (os.path.join(os.path.dirname(file), "decrypted_" + os.path.basename(file)))
                ), # because the encryption function turns the key into binary, unhexlify(key) would be called here to undo it
                number = times,
                repeat = repeat
            ))
        results.append((
            (mean(encrypt_trials), var(encrypt_trials, ddof = 1), std(encrypt_trials, ddof = 1) / sqrt(len(encrypt_trials))),
            (mean(decrypt_trials), var(decrypt_trials, ddof = 1), std(decrypt_trials, ddof = 1) / sqrt(len(decrypt_trials))),
        ))

# Compute mean of means
    mean_encrypt = mean([r[0][0] for r in results])
    mean_decrypt = mean([r[1][0] for r in results])

# Compute pooled variance
    var_encrypt = sum((repeat - 1) * r[0][1] for r in results) / sum(repeat - 1 for _ in results)
    var_decrypt = sum((repeat - 1) * r[1][1] for r in results) / sum(repeat - 1 for _ in results)

# Compute pooled standard error
    stderr_encrypt = sqrt(var_encrypt / (repeat * len(results)))
    stderr_decrypt = sqrt(var_decrypt / (repeat * len(results)))

    return (
        (mean_encrypt, var_encrypt, stderr_encrypt),
        (mean_decrypt, var_decrypt, stderr_decrypt)
    )

In [19]:
def print_results(test: str, result: tuple[tuple[float, float, float], tuple[float, float, float]]) -> None:
    print(f"Test {test}:\n\t")
    print(f"encryption:\n\t\tmean: {result[0][0]:.5f}\n\t\tstd error: {result[0][1]:.5e}\n\t\tvariance: {result[0][2]:.5e}")
    print(f"decryption:\n\t\tmean: {result[1][0]:.5f}\n\t\tstd error: {result[1][1]:.5e}\n\t\tvariance: {result[1][2]:.5e}")

In [20]:
sizes: int = [8, 64, 512, 4096, 32768, 262144, 2097152]
directory: str = 'random_files'

amount: int = 10
for size in sizes:
    for i in range(amount):
        filename: str = f'random_text_{size}_i.txt'
        write_to_file(directory, filename, size)

files: list[str] = [[directory + f"/random_text_{size}_i.txt" for i in range(amount)] for size in sizes]
for i in range(len(sizes)):
    print_results(sizes[i], test_aes(files[i], 100, 10))

File "random_text_8_i.txt" with size 8 bytes has been generated.
File "random_text_8_i.txt" with size 8 bytes has been generated.
File "random_text_8_i.txt" with size 8 bytes has been generated.
File "random_text_8_i.txt" with size 8 bytes has been generated.
File "random_text_8_i.txt" with size 8 bytes has been generated.
File "random_text_8_i.txt" with size 8 bytes has been generated.
File "random_text_8_i.txt" with size 8 bytes has been generated.
File "random_text_8_i.txt" with size 8 bytes has been generated.
File "random_text_8_i.txt" with size 8 bytes has been generated.
File "random_text_8_i.txt" with size 8 bytes has been generated.
File "random_text_64_i.txt" with size 64 bytes has been generated.
File "random_text_64_i.txt" with size 64 bytes has been generated.
File "random_text_64_i.txt" with size 64 bytes has been generated.
File "random_text_64_i.txt" with size 64 bytes has been generated.
File "random_text_64_i.txt" with size 64 bytes has been generated.
File "random_te

KeyboardInterrupt: 